In [ ]:
!pip install pandas-to-sql -U

In [ ]:
from copy import copy
import sqlite3
import pandas as pd
import pandas_to_sql
from pandas_to_sql import conventions

In [ ]:
iris = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv')
table_name = 'iris'
sql_connection = sqlite3.connect('./iris.db') #create db
iris.to_sql(table_name, sql_connection, if_exists='replace', index=False)
iris[:2]

In [ ]:
df = pandas_to_sql.wrap_df(iris, table_name)
pd_wrapped = pandas_to_sql.wrap_pd(pd)

In [ ]:
df.get_sql_string()

In [ ]:
species_petal_length_stats_df = df.groupby('species').agg({'petal_length':['mean','sum','count']})
species_petal_length_stats_df = conventions.flatten_grouped_dataframe(species_petal_length_stats_df)

species_petal_length_stats_df.get_sql_string()


In [ ]:
df[(df.petal_length>1.4) & (df.petal_width<.2)].get_sql_string()

In [ ]:
df_ = copy(df)
df_['sepal_width_rounded'] = df_.sepal_width.round()
df_1 = df_[df_.species=='setosa'].reset_index(drop=True)
df_2 = df_[df_.species=='versicolor'].reset_index(drop=True)
pd_wrapped.concat([df_1, df_2]).get_sql_string()

In [ ]:
df_ = copy(df)
df_['sepal_width_rounded'] = df_.sepal_width.round()
df_1 = df_[df_.species=='setosa'].reset_index(drop=True)
df_2 = df_[df_.species=='versicolor'].reset_index(drop=True)

some_df = pd_wrapped.concat([df_1, df_2])

sql_string = some_df.get_sql_string()

df_from_sql_database = pd.read_sql_query(sql_string, sql_connection)
df_pandas = some_df.df_pandas

from pandas_to_sql.testing.utils.asserters import assert_dataframes_equals
assert_dataframes_equals(df_pandas, df_from_sql_database)